## Train embedding representations using semi-supervised clustering

This notebook trains  embedding representations using semi-supervised models over the training dataset, initialized with pre-trained GloVe embeddings. In this context, "semi-supervised" means that in addition to using labeled data to initialize cluster centroids, we also force assignment of sentences with known labels to the corresponding cluster, thus greatly penalizing known-misclassified sentences in calculation of loss function.

This code has the following dependencies:
- training dataloaders stored in `data` folder, as generated by `data_prep/generate_dataloders.ipynb`
- custom functions found in `data_prep/generate_dataloaders.py`
- custom functions found in `evaluation/evaluation.py`

In [20]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

import pandas as pd
import random
import pickle as pkl
import datetime as dt
from tqdm import tqdm_notebook as tqdm
import os
import sys

# import scripts from data_prep & evaluation folders
path = os.getcwd()
parentdir = os.path.dirname(path)
sys.path.append(parentdir)
sys.path.append(parentdir + '/data_prep')
from generate_dataloaders import *
from evaluation import evaluation

import importlib
importlib.reload(evaluation)

<module 'evaluation.evaluation' from '/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/evaluation/evaluation.py'>

## Get Dataloaders

In [3]:
seed = 1029
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [4]:
path = os.getcwd()
data_dir = path + '/'
data_dir = path +'/data/' #Uncomment for local system

#### *Verify filenames are consistent*

In [5]:
train_loader = pkl.load(open(data_dir + 'train_dataloader.p','rb'))
train_loader_labelled = pkl.load(open(data_dir + 'train_labeled_dataloader.p','rb'))
train_loader_unlabelled = pkl.load(open(data_dir + 'train_unlabeled_dataloader.p','rb'))
val_loader = pkl.load(open(data_dir + 'val_dataloader.p','rb'))

In [6]:
review_dict = pkl.load(open(data_dir + 'dictionary.p','rb'))

In [7]:
#%conda install pytorch torchvision -c pytorch
## if torch.__version__ is not 1.3.1, run this cell then restart kernel

In [8]:
print(torch.__version__)

1.3.1


## PRE TRAINED WORD EMBEDDINGS 

In [9]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float16')

In [10]:
def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

In [11]:
def build_matrix(review_dict, embedding_index ,dim = 200):
#     embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(review_dict.tokens), dim))
    unknown_words = []
    
    for word, i in review_dict.ids.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [12]:
#glove_twitter = '../input/glove-global-vectors-for-word-representation/glove.twitter.27B.200d.txt' #Change loc for local system
glove_twitter = data_dir + 'glove.twitter.27B.200d.txt'

In [13]:
embedding_index = load_embeddings(glove_twitter)

In [14]:
glove_embedding_index,unknown_words = build_matrix(review_dict, embedding_index)
del embedding_index

In [15]:
len(review_dict.tokens)

16256

In [16]:
len(unknown_words)

4428

In [17]:
# for word in unknown_words:
#     print(word)

## Neural Network Class

NOTE: Data loader is defined as:
- tuple: (tokens, flagged_index, problematic)

In [18]:
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False
        
def unfreeze_model(model):
    for param in model.parameters():
        param.requires_grad = True

In [19]:
class neuralNetBow_glove(nn.Module):
    """
    BagOfWords classification model
    """
    # NOTE: we can't use linear layer until we take weighted average, otherwise it will
    # remember certain positions incorrectly (ie, 4th word has bigger weights vs 7th word)
    def __init__(self, opts):
        super(neuralNetBow_glove, self).__init__()
        self.embedding_matrix = opts['embedding_matrix']
        self.vocab_size = self.embedding_matrix.shape[0]
        self.embed_size = self.embedding_matrix.shape[1]
        self.upweight = opts['upweight']
        self.lambda_loss = opts['lambda_loss']
        
        self.embed = nn.Embedding(self.vocab_size, self.embed_size, padding_idx=0)
        self.embed.weight = nn.Parameter(torch.tensor(self.embedding_matrix,dtype=torch.float32))
        self.embed.weight.requires_grad = False
        
        self.upweight = upweight
    
    def forward(self, tokens, flagged_index):
        batch_size, num_tokens = tokens.shape
        embedding = self.embed(tokens)
        
        # upweight by flagged_index
        embedding[torch.LongTensor(range(batch_size)),flagged_index.type(torch.LongTensor),:] *= self.upweight
        
        # average across embeddings
        embedding_ave = embedding.sum(1) / (num_tokens + self.upweight - 1)
        
        return embedding_ave

### Clustering Stuff

In [20]:
class KMeansCriterion(nn.Module):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, embeddings, centroids, labelled = False,  cluster_assignments = None):
        if labelled:
            num_reviews = len(cluster_assignments)
            distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
            cluster_distances = distances[list(range(num_reviews)),cluster_assignments]
            loss = cluster_distances.sum()
        else:
            distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
            cluster_distances, cluster_assignments = distances.min(1)
            loss = cluster_distances.sum()
        return loss, cluster_assignments

In [21]:
def centroid_init(k, d, dataloader, model, current_device):
    ## Here we ideally don't want to do randomized/zero initialization
    centroid_sums = torch.zeros(k, d).to(current_device)
    centroid_counts = torch.zeros(k).to(current_device)
    for (tokens, labels, flagged_indices) in dataloader:
        # cluster_assignments = torch.LongTensor(tokens.size(0)).random_(k)
        cluster_assignments = labels.to(current_device)
        
        model.eval()
        sentence_embed = model(tokens.to(current_device),flagged_indices.to(current_device))
    
        update_clusters(centroid_sums, centroid_counts,
                        cluster_assignments, sentence_embed.to(current_device))
    
    centroid_means = centroid_sums / centroid_counts[:, None].to(current_device)
    return centroid_means.clone()

def update_clusters(centroid_sums, centroid_counts,
                    cluster_assignments, embeddings):
    k = centroid_sums.size(0)

    centroid_sums.index_add_(0, cluster_assignments, embeddings)
    bin_counts = torch.bincount(cluster_assignments,minlength=k).type(torch.FloatTensor).to(current_device)
    centroid_counts.add_(bin_counts)
    
    #np_cluster_assignments = cluster_assignments.to('cpu')
    #np_counts = np.bincount(np_cluster_assignments.data.numpy(), minlength=k)
    #centroid_counts.add_(torch.FloatTensor(np_counts))

## Dataloader stuff

In [23]:
def loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled):
    try:
        tokens, labels, flagged_indices = next(train_loader_labelled_iter)
    except StopIteration:
        train_loader_labelled_iter = iter(train_loader_labelled)
        tokens, labels, flagged_indices = next(train_loader_labelled_iter)

    return tokens, labels, flagged_indices, train_loader_labelled_iter


def loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled):
    try:
        tokens, labels, flagged_indices = next(train_loader_unlabelled_iter)
    except StopIteration:
        train_loader_unlabelled_iter = iter(train_loader_unlabelled)
        tokens, labels, flagged_indices = next(train_loader_unlabelled_iter)

    return tokens, labels, flagged_indices, train_loader_unlabelled_iter

## Training Function

In [52]:
def train_model(model, centroids, criterion, train_loader_labelled, train_loader_unlabelled, valid_loader, num_frozen_epochs=10, num_unfrozen_epochs=0, num_batches = 1000, path_to_save=None, print_every = 1000):

    train_loader_labelled_iter = iter(train_loader_labelled)
    train_loader_unlabelled_iter = iter(train_loader_unlabelled)
    lambda_loss = model.lambda_loss

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    # freeze part
    freeze_model(model)
    
    optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)
    
    for epoch in tqdm(range(num_frozen_epochs)):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.train()
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids).to(current_device)
        centroid_counts = torch.zeros(k).to(current_device)
        total_epoch_loss = 0
        
        for i in range(num_batches):
            tokens_labelled, labels, flagged_indices_labelled, train_loader_labelled_iter = loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled)
            tokens_unlabelled, _, flagged_indices_unlabelled, train_loader_unlabelled_iter = loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled)

            tokens_labelled = tokens_labelled.to(current_device)
            labels = labels.to(current_device)
            flagged_indices_labelled = flagged_indices_labelled.to(current_device)
            
            tokens_unlabelled = tokens_unlabelled.to(current_device)
            flagged_indices_unlabelled = flagged_indices_unlabelled.to(current_device)

            # forward pass and compute loss
            sentence_embed_labelled = model(tokens_labelled,flagged_indices_labelled)
            sentence_embed_unlabelled = model(tokens_unlabelled,flagged_indices_unlabelled)
            
            cluster_loss_unlabelled, cluster_assignments_unlabelled = criterion(sentence_embed_unlabelled, centroids.detach())
            cluster_loss_labelled, cluster_assignments_labelled = criterion(sentence_embed_labelled, centroids.detach(), labelled = True, cluster_assignments = labels)
    
            total_batch_loss = cluster_loss_unlabelled.data + lambda_loss * cluster_loss_labelled.data
        
            # run update step
            optimizer.zero_grad()
#             total_batch_loss.backward()
            optimizer.step()
            
            # Add loss to the epoch loss
            total_epoch_loss += total_batch_loss

#             # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums, centroid_counts,
                            cluster_assignments_labelled, sentence_embed_labelled)
    
            update_clusters(centroid_sums, centroid_counts,
                            cluster_assignments_unlabelled, sentence_embed_unlabelled)

            if i % print_every == 0:
                losses = total_batch_loss/(len(tokens_labelled)+ len(tokens_unlabelled))
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= (len(train_loader_labelled.dataset)+len(train_loader_unlabelled.dataset))
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1).to(current_device)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            #Add loss to the validation loss
            total_validation_loss += cluster_loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"embedding_matrix":model.embedding_matrix}
            torch.save(model.state_dict(), path_to_save+'model_dict.pt')
            torch.save(centroids, path_to_save+'centroids')
            torch.save(train_losses, path_to_save+'train_losses')
            torch.save(val_losses, path_to_save+'val_losses')
            torch.save(opts, path_to_save+'opts')
    
    # unfreeze part
    print("*** UNFREEZING MODEL ***")
    unfreeze_model(model)
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
    
    optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)
    
    for epoch in tqdm(range(num_unfrozen_epochs)):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.train()
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids).to(current_device)
        centroid_counts = torch.zeros(k).to(current_device)
        total_epoch_loss = 0
        
        for i in range(num_batches):
            tokens_labelled, labels, flagged_indices_labelled, train_loader_labelled_iter = loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled)
            tokens_unlabelled, _, flagged_indices_unlabelled, train_loader_unlabelled_iter = loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled)

            tokens_labelled = tokens_labelled.to(current_device)
            labels = labels.to(current_device)
            flagged_indices_labelled = flagged_indices_labelled.to(current_device)
            
            tokens_unlabelled = tokens_unlabelled.to(current_device)
            flagged_indices_unlabelled = flagged_indices_unlabelled.to(current_device)

            # forward pass and compute loss
            sentence_embed_labelled = model(tokens_labelled,flagged_indices_labelled)
            sentence_embed_unlabelled = model(tokens_unlabelled,flagged_indices_unlabelled)
            
            cluster_loss_unlabelled, cluster_assignments_unlabelled = criterion(sentence_embed_unlabelled, centroids.detach())
            cluster_loss_labelled, cluster_assignments_labelled = criterion(sentence_embed_labelled, centroids.detach(), labelled = True, cluster_assignments = labels)
    
            total_batch_loss = cluster_loss_unlabelled + lambda_loss * cluster_loss_labelled
        
            # run update step
            optimizer.zero_grad()
            total_batch_loss.backward()
            optimizer.step()
            
            # Add loss to the epoch loss
            total_epoch_loss += total_batch_loss.data

#             # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums.detach(), centroid_counts.detach(),
                            cluster_assignments_labelled.detach(), sentence_embed_labelled.detach())
    
            update_clusters(centroid_sums.detach(), centroid_counts.detach(),
                            cluster_assignments_unlabelled.detach(), sentence_embed_unlabelled.detach())

            if i % print_every == 0:
                losses = total_batch_loss/(len(tokens_labelled)+ len(tokens_unlabelled))
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= (len(train_loader_labelled.dataset)+len(train_loader_unlabelled.dataset))
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1).to(current_device)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            #Add loss to the validation loss
            total_validation_loss += cluster_loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        
        if path_to_save == None:
            pass
        else:
            opts = {"embedding_matrix":model.embedding_matrix}
            torch.save(model.state_dict(), path_to_save+'model_dict.pt')
            torch.save(centroids, path_to_save+'centroids')
            torch.save(train_losses, path_to_save+'train_losses')
            torch.save(val_losses, path_to_save+'val_losses')
            torch.save(opts, path_to_save+'opts')
            
        
    return model, centroids, train_losses, val_losses

In [53]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

## Hyperparameter Tuning Loop

In [54]:
def get_save_directory(opts):
    path = os.getcwd()
    model_folder = 'baseline_semisupervised_frozen_glove/'
    model_dir = path + '/models/' + model_folder
    
    # subfolder for each hyperparam config
    num_unfrozen_epochs = opts['num_unfrozen_epochs']
    upweight = opts['upweight']
    lambda_loss = opts['lambda_loss']
    subfolder = "num_unfrozen_epochs="+str(num_unfrozen_epochs) + ",upweight="+str(upweight) + ",lambda="+str(lambda_loss) + '/'
    
    # need to actually create these subfolders lol
    try:
        os.makedirs(model_dir + subfolder) # will throw error if subfolder already exists
    except:
        pass
    
    return model_dir + subfolder

In [55]:
def train_config(opts):
    model = neuralNetBow_glove(opts).to(current_device)
    num_unfrozen_epochs = opts['num_unfrozen_epochs']
    centroids = centroid_init(2, 200, train_loader_labelled, model, current_device)
    criterion = KMeansCriterion().to(current_device)
    path_to_save = get_save_directory(opts)
    print(path_to_save)
    
    train_model(model, centroids, criterion, train_loader_labelled, train_loader_unlabelled, val_loader, num_frozen_epochs=10, num_unfrozen_epochs=num_unfrozen_epochs, path_to_save=path_to_save)

In [72]:
num_unfrozen_epochs_list = [3, 4, 5]
upweights = [25]
lambda_losses = [.1, .5, 1, 5, 10, 25]

for num_unfrozen_epochs in num_unfrozen_epochs_list:
    for upweight in upweights:
        for lambda_loss in lambda_losses:
            opts = {
                'embedding_matrix': glove_embedding_index,
                'num_unfrozen_epochs': num_unfrozen_epochs,
                'upweight': upweight,
                'lambda_loss': lambda_loss
            }
            train_config(opts)

/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_semisupervised_frozen_glove/num_unfrozen_epochs=3,upweight=25,lambda=0.1/


2019-12-08 09:54:28.848303 | Epoch 0
Average training loss at batch  0 : 2.509
Average training loss after epoch  0 : 1.490
Average validation loss after epoch  0 : 4.796
2019-12-08 09:54:30.448626 | Epoch 1
Average training loss at batch  0 : 2.910
Average training loss after epoch  1 : 1.476
Average validation loss after epoch  1 : 4.788
2019-12-08 09:54:32.000215 | Epoch 2
Average training loss at batch  0 : 2.947
Average training loss after epoch  2 : 1.470
Average validation loss after epoch  2 : 4.787
2019-12-08 09:54:33.501372 | Epoch 3
Average training loss at batch  0 : 2.856
Average training loss after epoch  3 : 1.471
Average validation loss after epoch  3 : 4.787
2019-12-08 09:54:35.156478 | Epoch 4
Average training loss at batch  0 : 2.484
Average training loss after epoch  4 : 1.471
Average validation loss after epoch  4 : 4.787
2019-12-08 09:54:36.789977 | Epoch 5
Average training loss at batch  0 : 2.128
Average training loss after epoch  5 : 1.469
Average validation lo

2019-12-08 09:54:45.625110 | Epoch 0
Average training loss at batch  0 : 2.649
Average training loss after epoch  0 : 0.193
Average validation loss after epoch  0 : 0.189
2019-12-08 09:55:15.803976 | Epoch 1
Average training loss at batch  0 : 0.083
Average training loss after epoch  1 : 0.038
Average validation loss after epoch  1 : 0.076
2019-12-08 09:55:50.504594 | Epoch 2
Average training loss at batch  0 : 0.032
Average training loss after epoch  2 : 0.019
Average validation loss after epoch  2 : 0.049

/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_semisupervised_frozen_glove/num_unfrozen_epochs=3,upweight=25,lambda=0.5/


2019-12-08 09:56:30.533186 | Epoch 0
Average training loss at batch  0 : 3.684
Average training loss after epoch  0 : 2.095
Average validation loss after epoch  0 : 4.795
2019-12-08 09:56:32.276397 | Epoch 1
Average training loss at batch  0 : 2.917
Average training loss after epoch  1 : 2.070
Average validation loss after epoch  1 : 4.788
2019-12-08 09:56:33.998025 | Epoch 2
Average training loss at batch  0 : 3.339
Average training loss after epoch  2 : 2.063
Average validation loss after epoch  2 : 4.786
2019-12-08 09:56:35.476260 | Epoch 3
Average training loss at batch  0 : 3.425
Average training loss after epoch  3 : 2.066
Average validation loss after epoch  3 : 4.787
2019-12-08 09:56:37.025540 | Epoch 4
Average training loss at batch  0 : 3.670
Average training loss after epoch  4 : 2.068
Average validation loss after epoch  4 : 4.787
2019-12-08 09:56:38.578993 | Epoch 5
Average training loss at batch  0 : 3.027
Average training loss after epoch  5 : 2.059
Average validation lo

2019-12-08 09:56:46.724726 | Epoch 0
Average training loss at batch  0 : 3.814
Average training loss after epoch  0 : 0.243
Average validation loss after epoch  0 : 0.159
2019-12-08 09:57:17.509396 | Epoch 1
Average training loss at batch  0 : 0.095
Average training loss after epoch  1 : 0.041
Average validation loss after epoch  1 : 0.063
2019-12-08 09:57:53.434213 | Epoch 2
Average training loss at batch  0 : 0.035
Average training loss after epoch  2 : 0.020
Average validation loss after epoch  2 : 0.043

/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_semisupervised_frozen_glove/num_unfrozen_epochs=3,upweight=25,lambda=1/


...

2019-12-08 10:33:16.291645 | Epoch 0
Average training loss at batch  0 : 12.767
Average training loss after epoch  0 : 1.067
Average validation loss after epoch  0 : 0.173
2019-12-08 10:33:45.407496 | Epoch 1
Average training loss at batch  0 : 0.430
Average training loss after epoch  1 : 0.155
Average validation loss after epoch  1 : 0.079
2019-12-08 10:34:20.225519 | Epoch 2
Average training loss at batch  0 : 0.300
Average training loss after epoch  2 : 0.065
Average validation loss after epoch  2 : 0.048
2019-12-08 10:34:59.569746 | Epoch 3
Average training loss at batch  0 : 0.053
Average training loss after epoch  3 : 0.037
Average validation loss after epoch  3 : 0.039
2019-12-08 10:35:40.415587 | Epoch 4
Average training loss at batch  0 : 0.034
Average training loss after epoch  4 : 0.026
Average validation loss after epoch  4 : 0.034

/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_semisupervised_frozen_glove/num_unfrozen_epochs=5,upweight=25,lambda=10/


2019-12-08 10:36:21.842378 | Epoch 0
Average training loss at batch  0 : 27.761
Average training loss after epoch  0 : 16.208
Average validation loss after epoch  0 : 4.796
2019-12-08 10:36:23.322977 | Epoch 1
Average training loss at batch  0 : 32.895
Average training loss after epoch  1 : 16.247
Average validation loss after epoch  1 : 4.787
2019-12-08 10:36:24.847643 | Epoch 2
Average training loss at batch  0 : 24.863
Average training loss after epoch  2 : 16.243
Average validation loss after epoch  2 : 4.785
2019-12-08 10:36:26.414326 | Epoch 3
Average training loss at batch  0 : 25.417
Average training loss after epoch  3 : 16.272
Average validation loss after epoch  3 : 4.786
2019-12-08 10:36:27.960201 | Epoch 4
Average training loss at batch  0 : 26.184
Average training loss after epoch  4 : 16.283
Average validation loss after epoch  4 : 4.787
2019-12-08 10:36:29.473248 | Epoch 5
Average training loss at batch  0 : 21.863
Average training loss after epoch  5 : 16.256
Average v

2019-12-08 10:36:37.160392 | Epoch 0
Average training loss at batch  0 : 31.384
Average training loss after epoch  0 : 2.046
Average validation loss after epoch  0 : 0.182
2019-12-08 10:37:06.278408 | Epoch 1
Average training loss at batch  0 : 1.052
Average training loss after epoch  1 : 0.290
Average validation loss after epoch  1 : 0.086
2019-12-08 10:37:41.501326 | Epoch 2
Average training loss at batch  0 : 0.294
Average training loss after epoch  2 : 0.119
Average validation loss after epoch  2 : 0.057
2019-12-08 10:38:21.897679 | Epoch 3
Average training loss at batch  0 : 0.130
Average training loss after epoch  3 : 0.067
Average validation loss after epoch  3 : 0.049
2019-12-08 10:39:02.760817 | Epoch 4
Average training loss at batch  0 : 0.081
Average training loss after epoch  4 : 0.046
Average validation loss after epoch  4 : 0.041

/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_semisupervised_frozen_glove/num_unfrozen_epochs=5,upweight=25,lambda=25/


2019-12-08 10:39:49.587000 | Epoch 0
Average training loss at batch  0 : 67.976
Average training loss after epoch  0 : 38.505
Average validation loss after epoch  0 : 4.796
2019-12-08 10:39:51.246073 | Epoch 1
Average training loss at batch  0 : 76.188
Average training loss after epoch  1 : 38.585
Average validation loss after epoch  1 : 4.789
2019-12-08 10:39:52.888197 | Epoch 2
Average training loss at batch  0 : 65.935
Average training loss after epoch  2 : 38.702
Average validation loss after epoch  2 : 4.787
2019-12-08 10:39:54.543926 | Epoch 3
Average training loss at batch  0 : 72.237
Average training loss after epoch  3 : 38.703
Average validation loss after epoch  3 : 4.787
2019-12-08 10:39:56.165805 | Epoch 4
Average training loss at batch  0 : 58.349
Average training loss after epoch  4 : 38.636
Average validation loss after epoch  4 : 4.787
2019-12-08 10:39:57.851103 | Epoch 5
Average training loss at batch  0 : 54.617
Average training loss after epoch  5 : 38.723
Average v

2019-12-08 10:40:06.209818 | Epoch 0
Average training loss at batch  0 : 67.315
Average training loss after epoch  0 : 4.996
Average validation loss after epoch  0 : 0.217
2019-12-08 10:40:36.672927 | Epoch 1
Average training loss at batch  0 : 1.709
Average training loss after epoch  1 : 0.698
Average validation loss after epoch  1 : 0.088
2019-12-08 10:41:12.278258 | Epoch 2
Average training loss at batch  0 : 0.567
Average training loss after epoch  2 : 0.279
Average validation loss after epoch  2 : 0.060
2019-12-08 10:41:51.708104 | Epoch 3
Average training loss at batch  0 : 0.220
Average training loss after epoch  3 : 0.157
Average validation loss after epoch  3 : 0.051
2019-12-08 10:42:32.560197 | Epoch 4
Average training loss at batch  0 : 0.122
Average training loss after epoch  4 : 0.106
Average validation loss after epoch  4 : 0.040



# Evaluate Model

In [70]:
pd.options.display.max_rows = 100

In [57]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

In [62]:
def evaluate_config(opts,verbose=True):
    path_to_save = get_save_directory(opts)
    #print(path_to_save)
    
    model = neuralNetBow_glove(opts) #change according to model inputs
    model.load_state_dict(torch.load(path_to_save+'model_dict.pt',map_location=lambda storage, loc: storage))
    model = model.to(current_device)
    criterion = KMeansCriterion().to(current_device)
    centroids = torch.load(path_to_save+'centroids',map_location=lambda storage, loc: storage)
    
    TP_cluster, FP_cluster, results_dict = evaluation.main(model, centroids, val_loader, criterion, data_dir, current_device, verbose)
    results_dict.update(opts)
    return TP_cluster, FP_cluster, results_dict

In [73]:
num_unfrozen_epochs_list = [0, 1, 2]
upweights = [1, 5, 10, 25]
lambda_losses = [.1, .5, 1, 5, 10, 25]

results_df = pd.DataFrame()
for num_unfrozen_epochs in num_unfrozen_epochs_list:
    for upweight in upweights:
        for lambda_loss in lambda_losses:
            opts = {
                'embedding_matrix': glove_embedding_index,
                'num_unfrozen_epochs': num_unfrozen_epochs,
                'upweight': upweight,
                'lambda_loss': lambda_loss
            }
            _, _, results_dict = evaluate_config(opts,False)
            results_df = results_df.append(results_dict,ignore_index=True)
      
# tried just 25 for the following:
num_unfrozen_epochs_list = [3, 4, 5]
upweights = [25]
lambda_losses = [.1, .5, 1, 5, 10, 25]
for num_unfrozen_epochs in num_unfrozen_epochs_list:
    for upweight in upweights:
        for lambda_loss in lambda_losses:
            opts = {
                'embedding_matrix': glove_embedding_index,
                'num_unfrozen_epochs': num_unfrozen_epochs,
                'upweight': upweight,
                'lambda_loss': lambda_loss
            }
            _, _, results_dict = evaluate_config(opts,False)
            results_df = results_df.append(results_dict,ignore_index=True)
        
results_df = results_df[['num_unfrozen_epochs','upweight','lambda_loss','Accuracy','F1 score','Precision','Recall',
                        'TP_rate','FP_rate','FN_rate','TN_rate']]

In [74]:
results_df

,num_unfrozen_epochs,upweight,lambda_loss,Accuracy,F1 score,Precision,Recall,TP_rate,FP_rate,FN_rate,TN_rate
0,0.0,1.0,0.1,0.564214,0.677387,0.915014,0.537738,0.915014,0.084986,0.786585,0.213415
1,0.0,1.0,0.5,0.564214,0.677387,0.915014,0.537738,0.915014,0.084986,0.786585,0.213415
2,0.0,1.0,1.0,0.564214,0.677387,0.915014,0.537738,0.915014,0.084986,0.786585,0.213415
3,0.0,1.0,5.0,0.564214,0.677387,0.915014,0.537738,0.915014,0.084986,0.786585,0.213415
4,0.0,1.0,10.0,0.564214,0.677387,0.915014,0.537738,0.915014,0.084986,0.786585,0.213415
5,0.0,1.0,25.0,0.564989,0.677833,0.915254,0.538217,0.915254,0.084746,0.785276,0.214724
6,0.0,5.0,0.1,0.560509,0.674914,0.912429,0.535513,0.912429,0.087571,0.791411,0.208589
7,0.0,5.0,0.5,0.560509,0.674914,0.912429,0.535513,0.912429,0.087571,0.791411,0.208589
8,0.0,5.0,1.0,0.560509,0.674914,0.912429,0.535513,0.912429,0.087571,0.791411,0.208589
9,0.0,5.0,5.0,0.560509,0.674914,0.912429,0.535513,0.912429,0.087571,0.791411,0.208589


In [69]:
# best params: num_unfrozen=3, upweight=25, lambda_loss = 1

In [81]:
results_df.sort_values(['F1 score'],ascending=False)

,num_unfrozen_epochs,upweight,lambda_loss,Accuracy,F1 score,Precision,Recall,TP_rate,FP_rate,FN_rate,TN_rate
74,3.0,25.0,1.0,0.784268,0.812844,0.936957,0.717767,0.936957,0.063043,0.368421,0.631579
68,2.0,25.0,1.0,0.782992,0.812256,0.938865,0.715737,0.938865,0.061135,0.372881,0.627119
44,1.0,25.0,1.0,0.775541,0.806059,0.932900,0.709582,0.932900,0.067100,0.381818,0.618182
69,2.0,25.0,5.0,0.757811,0.795498,0.942094,0.688381,0.942094,0.057906,0.426471,0.573529
85,5.0,25.0,0.5,0.753663,0.793044,0.943946,0.683739,0.943946,0.056054,0.436620,0.563380
45,1.0,25.0,5.0,0.753591,0.792650,0.941964,0.684196,0.941964,0.058036,0.434783,0.565217
80,4.0,25.0,1.0,0.753776,0.791750,0.936123,0.685958,0.936123,0.063877,0.428571,0.571429
73,3.0,25.0,0.5,0.749688,0.790382,0.943820,0.679856,0.943820,0.056180,0.444444,0.555556
43,1.0,25.0,0.5,0.749688,0.790382,0.943820,0.679856,0.943820,0.056180,0.444444,0.555556
70,2.0,25.0,10.0,0.749688,0.790382,0.943820,0.679856,0.943820,0.056180,0.444444,0.555556


In [83]:
results_df[results_df.num_unfrozen_epochs==0].sort_values(['F1 score'],ascending=False)

,num_unfrozen_epochs,upweight,lambda_loss,Accuracy,F1 score,Precision,Recall,TP_rate,FP_rate,FN_rate,TN_rate
20,0.0,25.0,1.0,0.627055,0.722114,0.969136,0.575441,0.969136,0.030864,0.715026,0.284974
18,0.0,25.0,0.1,0.627055,0.722114,0.969136,0.575441,0.969136,0.030864,0.715026,0.284974
23,0.0,25.0,25.0,0.620993,0.718704,0.968354,0.571394,0.968354,0.031646,0.726368,0.273632
21,0.0,25.0,5.0,0.619544,0.717890,0.968153,0.570436,0.968153,0.031847,0.729064,0.270936
19,0.0,25.0,0.5,0.618121,0.717090,0.967949,0.569497,0.967949,0.032051,0.731707,0.268293
22,0.0,25.0,10.0,0.616030,0.715916,0.967638,0.568124,0.967638,0.032362,0.735577,0.264423
13,0.0,10.0,0.5,0.565596,0.678531,0.916905,0.538526,0.916905,0.083095,0.785714,0.214286
17,0.0,10.0,25.0,0.565596,0.678531,0.916905,0.538526,0.916905,0.083095,0.785714,0.214286
16,0.0,10.0,10.0,0.565596,0.678531,0.916905,0.538526,0.916905,0.083095,0.785714,0.214286
15,0.0,10.0,5.0,0.565596,0.678531,0.916905,0.538526,0.916905,0.083095,0.785714,0.214286


In [86]:
results_df[(results_df.num_unfrozen_epochs==3) & (results_df.upweight == 25)].sort_values(['F1 score'],ascending=False)

,num_unfrozen_epochs,upweight,lambda_loss,Accuracy,F1 score,Precision,Recall,TP_rate,FP_rate,FN_rate,TN_rate
74,3.0,25.0,1.0,0.784268,0.812844,0.936957,0.717767,0.936957,0.063043,0.368421,0.631579
73,3.0,25.0,0.5,0.749688,0.790382,0.943820,0.679856,0.943820,0.056180,0.444444,0.555556
75,3.0,25.0,5.0,0.734168,0.779784,0.941309,0.665574,0.941309,0.058691,0.472973,0.527027
77,3.0,25.0,25.0,0.719388,0.769874,0.938776,0.652482,0.938776,0.061224,0.500000,0.500000
76,3.0,25.0,10.0,0.700812,0.757404,0.934091,0.636927,0.934091,0.065909,0.532468,0.467532
72,3.0,25.0,0.1,0.639110,0.727416,0.963068,0.584416,0.963068,0.036932,0.684848,0.315152
